<a href="https://colab.research.google.com/github/YunhoCha/2024-DH/blob/main/gb_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1_웹 스크래핑

### 학습 목표
- 국사편찬위원회 한국사데이터베이스의 사료를 자유롭게 크롤링하고 수정하여 나만의 DB를 만들 수 있다.
- 형태소분석기를 활용해 사료 텍스트에서 개념어를 추출하고 빈도 수를 분석해본다.
- 위 과정에 활용되는 프로그래밍 언어의 작동 원리를 이해하고, chatGPT라는 보조도구를 활용해 평소에도 써본다.

**주의**
- 시작전 왼쪽에 data 폴더와 result 폴더을 만들고 data 폴더에 메타데이터 파일을 꼭 넣을 것

# 2_필요한 도구와 라이브러리 설치

In [ ]:
import ssl
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 3_url 분석

In [ ]:
# "개벽" 기사가 서비스되는 웹페이지 주소 1개를 불러와서 url 변수에 저장하기
# 이 url에서 기사별 id가 있는 부분을 경계로 그 이전과 이후를 분리하기

url = 'https://db.history.go.kr/modern/level.do?levelId=ma_013_0010_0001'
url_s = url.split('ma_013_0010_0001')
print(url)
print(url_s)

# 4_기사정보 다운로드

사전 작업
- 국편 한국사데이터베이스/개벽에서 기사정보 다운로드
- '개벽_기사정보.txt'로 저장

In [ ]:
# 이름 조심하기
gb_info = 'data/근현대잡지자료 수정본.txt'

In [ ]:
df = pd.read_csv(gb_info, delimiter='^', encoding='utf-8')
df

In [ ]:
df.info()

In [ ]:
# '발행일' 열이 공백이면 기사내용이 포함되지 않은 행이라는 뜻
df1 = df[['자료ID', '기사제목', '필자', '기사형태', '발행일', 'URL']]
df1

In [ ]:
# 결측치 확인, 발행일열이 공백인 77개의 행을 없앨 필요가 있음
df1.isna().sum()

In [ ]:
# 발행일 열이 공백인 행을 보기
df1[df1['발행일'].isna()][:30]

In [ ]:
# 요약통계
df1.describe()

In [ ]:
# 값 파악
df1['발행일'].value_counts().sort_index()[60:90]

In [ ]:
# 결측치 제거
df1 = df1.copy()
df1 = df1.dropna(subset = ['발행일'])  # 발행일 열이 비어있는 곳을 없애기
df1

In [ ]:
# 1930년대 개벽 제외
df1['year'] = df1['발행일'].apply(lambda x: int(x[:4]))
df2 = df1.query('year < 1930')
df2

In [ ]:
df2.info() # 자료id가 2257개 임을 알 수 있음

In [ ]:
df2.isna().sum() # 다시한번 결측치를 확인하니 발행일 쪽 결측치가 0이 됨

In [ ]:
# 자료ID 길이 확인 <- 크로스 체크
df2['자료ID'].str.len().value_counts()
# 자료id가 글자길이가 16인 것이 2257개임을 알 수 있음
# 만약 글자길이가 다른 id가 존재한다면 그것도 결측치로 제거할 수 있음(밑 코드 이용)

In [ ]:
# df2 = df1[df1['자료ID'].str.len() != 17] # 길이가 17인 자료id 행 삭제
# df2

# 5_기사 id를 이용한 원문 스크래핑
- url_s 변수를 이용하여 기사 ID를 포함한 웹페이지 주소를 생성
- 이 주소를 이용하여 웹 페이지에 접속하고, BeautifulSoup을 사용하여 원하는 부분의 데이터를 추출
- 국편위 개별 사료 페이지로 가서 Ctrl + Shift + i 를 누르기, html에서 텍스트가 어느 위치에 있는지 알아보기(실습)

In [ ]:
# 기사 id를 모두 리스트로 변환

id_List = df2['자료ID'].to_list()
print(len(id_List))
id_List[:10]

In [ ]:
# 본문 가져오기 사용자 함수 작성

def get_contents(url):
    nm = 1
    results = []  # 결과를 저장할 리스트 생성

    # SSL 인증서 확인을 건너뛰도록 설정
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    for ids in id_List[:5]: # 샘플로 5개 기사만 추출, 숫자를 아까 자료 id 전체 크기로 바꾸면 모든 사료를 스크래핑하는 것
        N_url = url_s[0] + ids + url_s[-1]
        # SSL 인증서 확인을 건너뛰는 urlopen 사용
        webpage = urlopen(N_url, context=ctx)

        bsobj = BeautifulSoup(webpage.read(), 'html.parser')
        List1 = bsobj.findAll('div', {'id': 'cont_view'})  # 이 부분을 국편위 사이트 개별 사료에서 html을 파악하여 위치를 넣는 곳

        for z in List1:
            z1 = z.get_text('\n', strip=True)
            results.append([ids, z1])  # 결과를 리스트에 추가

    # pandas DataFrame으로 변환
    result_df = pd.DataFrame(results, columns=['자료ID', 'Content'])
    return result_df

In [ ]:
contents_5_df = get_contents(url)
contents_5_df

In [ ]:
# 본문 내용 확인 (첫 행)
print(contents_5_df.loc[3,'Content'])

# 6_데이터 저장

In [ ]:
# 두 데이터프레임을 '자료ID' 기준으로 통합

combi_df = pd.merge(df2, contents_5_df, on='자료ID', how='inner')
combi_df

In [ ]:
# 저장
combi_df.to_excel('result/A1_raw5.xlsx', index=False)  # 인덱스를 저장하지 않도록 설정

# The End of Note

---

# 텍스트 전처리(형태소 분석)과 간단한 빈도 분석 실습

## 1. 형태소 분석(Kiwi 형태소 분석기)

### 1) 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import re

In [ ]:
# 설정된 경로에서 엑셀 파일 불러오기
rawdata = 'result/A1_raw5.xlsx'

In [ ]:
# 데이터 불러오기

df = pd.read_excel(rawdata)
print(df.shape)

df1 = df[:30] # 기사 30개만 샘플로
print(df1.shape)
df1[:3] # 그중 3개만 보여주기

### 2) 기사를 문장별로 분리하기. - 파일을 보고 어떻게 나누는게 좋을지 고민하기

In [ ]:
# 기사를 문장별로 분리하는 사용자 함수
separators = ['\n', '.', '!', '?'] # 엔터 및 문장 종결 부호

def split_content_by_separators(content, separators):
    """Splits content based on the given separators."""
    escaped_separators = [re.escape(sep) for sep in separators]  # separators = 구분자들
    return re.split('|'.join(escaped_separators), content)

In [ ]:
# 사용자 함수 적용하여 문장별로 분리하여 행으로 재구성

new_df = df1.apply(lambda row: pd.Series(split_content_by_separators(row['Content'], separators)), axis=1) \
            .stack() \
            .reset_index(level=1, drop=True) \
            .to_frame(name='Content') \
            .join(df1[['자료ID']], how='left')

new_df = new_df[['자료ID', 'Content']]
print(new_df.shape)
new_df

In [ ]:
# 문장별 id 생성 ##### 중요 !!
new_sent_df = new_df.reset_index(drop=True)
new_sent_df['sent_id'] = new_sent_df.index + 1
new_sent_df = new_sent_df[['sent_id', '자료ID', 'Content']]
new_sent_df
# 320개의 문장이 있다는것을 알 수 있음

### 3) 형태소 분석
- 제목: 형태소 분석기 적용
- kiwi 형태소 분석기로 텍스트 데이터 분석, 각 언어 별로 형태소 분석기가 있음 - 분석기별 코드는 chatCPT로 만들어보자.
- 한국어 형태소 분석기(kiwi)에서 제공하는 서브워드(subword) 학습 함수를 활용해 1920~40년에 출간된 조선/동아일보 약 190만건의 기사를 학습 후 만든 근대 한국어 토크나이저.(카이스트 김병준) https://github.com/ByungjunKim/ModernKoreanSubword
- cf) 유튜브 동영상 https://www.youtube.com/watch?v=5VmKXYwnYeQ

In [ ]:
# kiwi 설치
!pip install -U kiwipiepy

In [ ]:
# 서브워드 토크나이저 json 파일 다운로드
!git clone https://github.com/ByungjunKim/ModernKoreanSubword

In [ ]:
# kiwi 로드
from kiwipiepy import Kiwi
kiwi = Kiwi()
from kiwipiepy.sw_tokenizer import SwTokenizer

# 토크나이저 설정
tokenizer = SwTokenizer('./ModernKoreanSubword/ModernKoreanSubword.json', kiwi=kiwi)

In [ ]:
# 1음절 한자 이어질 때 두 개를 합해 줌 - 아직 한국 한문의 형태소를 분석해주는 분석기는 없음. 따라서 나만의 기준으로 한자 개념어를 추출해야 함.
# 여기서는 간단히 한자를 하나씩 뽑아내서 2개씩 합치는 형태로 진행.

def is_hanja(ch):
    # 한자의 범위는 U+4E00 부터 U+9FFF 입니다.
    return '\u4e00' <= ch <= '\u9fff'

def combine_one_hanja(lst):
    result = []
    i = 0
    while i < len(lst):
        if (len(lst[i]) == 1 and i + 1 < len(lst) and len(lst[i + 1]) == 1 and
        is_hanja(lst[i]) and is_hanja(lst[i + 1])):
            result.append(lst[i] + lst[i + 1])
            i += 2
        else:
            result.append(lst[i])
            i += 1
    return result

In [ ]:
# 영어, 숫자, 특정 부호 등 제외하기

def filter_elements(lst):
    # 영어, 숫자 및 '<' 문자만을 허용하는 정규식 패턴
    pattern = re.compile(r'^[a-zA-Z0-9<＞「」]+$')

    # 해당 패턴과 일치하지 않는 문자열만을 반환하는 필터링 함수
    return list(filter(lambda x: not pattern.match(x), lst))

In [ ]:
# 본격 형태소 분석

import string
def do_morph(sent):
    # 토큰나이징 결과 확인 (subword + 한국어 형태소)
    vocab_list = tokenizer.encode(sent)
    result01 = [tokenizer.id2vocab[i].replace('#', '') for i in vocab_list if '/' not in tokenizer.id2vocab[i]]
    result01 = [''.join([char for char in word if char not in string.punctuation]) for word in result01]
    result01 = [word for word in result01 if len(word) > 0] # 공백 제외
    result01 = combine_one_hanja(result01) # 연속되는 1음절 한자 합침 (사용자 함수)
    result01 = filter_elements(result01) # 영어, 숫자, 특정부호 제외 (사용자 함수)

    return result01

In [ ]:
# 자료 확인해보기
new_sent_df[:3]

In [ ]:
# 'Content' 열에 kiwi 형태소 분석기 적용
new_sent_df['tokens'] = new_sent_df['Content'].apply(do_morph)
new_sent_df

In [ ]:
# 'tokens' 열에서 리스트 요소가 하나도 없는 행 제거
new_sent_df1 = new_sent_df[new_sent_df['tokens'].str.len() > 0]

new_sent_df1 = new_sent_df1.copy()
new_sent_df1['tokens'] = new_sent_df1['tokens'].apply(lambda x: ' '.join(x)) # 리스트를 문자열로

new_sent_df1.reset_index(drop=True, inplace=True) # 인덱스 초기화
new_sent_df1['sent_id'] = new_sent_df1.index + 1 # sent_id 재넘버링
new_sent_df1.head()

## 2. 단어 빈도 분석 및 chatGPT로 연도별 개념어 빈도 분석해보기

In [ ]:
# 'tokens' 열의 문자열을 빈칸으로 분리하여 모든 단어들을 리스트로 추출
words = new_sent_df1['tokens'].str.split().explode()
words

In [ ]:
# 단어들의 빈도 계산
word_counts = words.value_counts()

# 결과 출력
print(word_counts[:20])

In [ ]:
# 인덱스(단어)의 길이가 2 이상인 것만 선택
word_counts_1 = word_counts[word_counts.index.str.len() >= 2]
word_counts_1[:20]

In [ ]:
new_sent_df1.to_excel('result/A2.xlsx', index=False)  # 인덱스를 저장하지 않도록 설정

## result 폴더에서 우리가 만든 파일 두 개를 다운받아 보자. 그리고 chatGPT에 업로드하여 다음과 같이 질문해보자.

"a1은 기사를 스크래핑 한 것이고, a2에는 문장별로 기사의 내용을 나누어서 형태소 분석기로 단어를 추출해놓은 것까지 진행한 것이야. 두 파일을 연동하여 연도별로 2글자 이상의 개념어의 빈도수를 알고 싶어"

"그리고 각 연도별 가장 많이 나온 단어 top3를 알려줘"